# INPUT

In [1]:
grid = []
with open('input.txt') as f:
    for line in f:
        grid.append(list(line.strip()))


# PART 1

In [2]:
def check(grid, x, y, dim_x, dim_y):
    counter = 0
    for x_loc in range(max(0, x-1), min(dim_x - 1, x + 1)+1):
        for y_loc in range(max(0, y-1), min(dim_y - 1, y + 1)+1):
            if grid[x_loc][y_loc] == '@':
                counter+=1
                if counter == 5:
                    return 0
    return 1 

In [3]:
dim_x = len(grid[0])
dim_y = len(grid)

forkable = 0
for x in range(0, dim_x):
    for y in range(0, dim_y):
        if grid[x][y] == '@':
                forkable += check(grid, x, y, dim_x, dim_y)


forkable


1626

# PART 2

In [4]:
dim_x = len(grid[0])
dim_y = len(grid)

forkable = 0
forked_iteration = 0
iter_one = True
new_grid = grid
while forked_iteration > 0 or iter_one:
    iter_one = False
    forked_iteration = 0
    grid = new_grid
    for x in range(0, dim_x):
        for y in range(0, dim_y):
            if grid[x][y] == '@':
                can_be_removed = check(grid, x, y, dim_x, dim_y)
                forked_iteration += can_be_removed
                if can_be_removed == 1:
                    new_grid[x][y] = '.'
                else:
                    new_grid[x][y] = grid[x][y]

    forkable +=forked_iteration


forkable

9173

# EKSTRA

In [21]:
import torch

grid = []
with open('input.txt') as f:
    for line in f:
        grid.append([-1 if ch == '.' else 1 for ch in list(line.strip())])

tensor_data = torch.tensor(grid, dtype=torch.float32).unsqueeze(0).unsqueeze(0)

in_channels = 1
out_channels = 1
kernel_size = 3
padding = 1

conv_no_center = torch.nn.Conv2d(in_channels, out_channels, kernel_size, padding=padding, bias=False)

custom_kernel = torch.tensor([
    [1./8, 1./8, 1./8],
    [1./8, 0.  , 1./8],
    [1./8, 1./8, 1./8]
], dtype=torch.float32).view(out_channels, in_channels, kernel_size, kernel_size)

with torch.no_grad():
    conv_no_center.weight.copy_(custom_kernel)


output_tensor = torch.clamp(conv_no_center(tensor_data).squeeze(),0,1,)
print(output_tensor)
torch.count_nonzero(output_tensor).item()

tensor([[0.0000, 0.1250, 0.0000,  ..., 0.6250, 0.0000, 0.1250],
        [0.1250, 0.0000, 0.2500,  ..., 0.7500, 0.2500, 0.3750],
        [0.0000, 0.0000, 0.0000,  ..., 0.2500, 0.2500, 0.1250],
        ...,
        [0.6250, 0.7500, 0.7500,  ..., 0.7500, 0.2500, 0.1250],
        [0.3750, 0.7500, 0.7500,  ..., 0.0000, 0.2500, 0.3750],
        [0.3750, 0.6250, 0.3750,  ..., 0.1250, 0.0000, 0.0000]],
       grad_fn=<ClampBackward1>)


13599